In [1]:
! pip install pymongo
! pip install -U plotly
! pip install squarify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 37.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.io as pio
pio.renderers.default = "notebook_connected"  # opción segura en Colab

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

%matplotlib inline

In [3]:
# Datos de conexión
usuario = "madris1402"
password = "maranes8"
cluster = "clusterrmu.s1ici9p.mongodb.net"
base_datos = "analytics"
coleccion = "sales_cubo"

# Cadena de conexión
uri = f"mongodb+srv://{usuario}:{password}@{cluster}/?retryWrites=true&w=majority"
# Crear cliente
client = MongoClient(uri)

#Madrigal Urencio Ricardo
client

#Conexion a la DB
db = client[base_datos]
coll = db[coleccion]

print(db)
print(coll)

Database(MongoClient(host=['ac-tljpgji-shard-00-02.s1ici9p.mongodb.net:27017', 'ac-tljpgji-shard-00-01.s1ici9p.mongodb.net:27017', 'ac-tljpgji-shard-00-00.s1ici9p.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-10ra0l-shard-0', tls=True), 'analytics')
Collection(Database(MongoClient(host=['ac-tljpgji-shard-00-02.s1ici9p.mongodb.net:27017', 'ac-tljpgji-shard-00-01.s1ici9p.mongodb.net:27017', 'ac-tljpgji-shard-00-00.s1ici9p.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-10ra0l-shard-0', tls=True), 'analytics'), 'sales_cubo')


In [5]:
# Obtener valores únicos desde la base de datos
gender_options = sorted(coll.distinct("gender"))
continent_options = sorted(coll.distinct("continent"))

continent_options

['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']

In [8]:
# Output contenedor para el gráfico
graph_output = widgets.Output()

# Filtros
gender_dropdown = widgets.SelectMultiple(
    options=gender_options,
    value=['Male', 'Female'],
    description='Género:'
)

continent_dropdown = widgets.Dropdown(
    options=continent_options,
    value='Africa',
    description='Continente:'
)

year_range = widgets.IntRangeSlider(
    value=[2020, 2022],
    min=2015,
    max=2025,
    step=1,
    description='Años:',
    continuous_update=False
)

button = widgets.Button(description="Actualizar Gráfico")

In [9]:
# Función de actualización
def update_graph(b):
    with graph_output:
        clear_output(wait=True)

        pipeline = [
            { "$match": {
                "continent": { "$regex": continent_dropdown.value, "$options": "i" },
                "gender": { "$in": list(gender_dropdown.value) },
                "year": { "$gte": year_range.value[0], "$lte": year_range.value[1] }
            }},
            { "$group": {
                "_id": { "country": "$country_name", "gender": "$gender" },
                "avg_age": { "$avg": "$age" }
            }},
            { "$project": {
                "country_name": "$_id.country",
                "gender": "$_id.gender",
                "avg_age": { "$round": ["$avg_age", 2] },
                "_id": 0
            }},
            { "$sort": { "country_name": 1, "gender": 1 } }
        ]

        result = list(coll.aggregate(pipeline))
        df = pd.DataFrame(result)

        if not df.empty:
            fig = px.bar(df,
                         x='country_name',
                         y='avg_age',
                         color='gender',
                         barmode='group',
                         title=f'Edad Promedio por País y Género, {continent_dropdown.value}')
            fig.update_layout(transition_duration=500)
            fig.show()
        else:
            print("Sin datos para los filtros seleccionados.")

In [11]:
# Conectar botón
button.on_click(update_graph)

In [21]:
display(widgets.VBox([gender_dropdown, continent_dropdown, year_range, button, graph_output]))